In [1]:
%load_ext autoreload
%autoreload 2  # Autoreload all modules

In [2]:
import numpy as np
import matplotlib.pylab as mpl
from volumenes_finitos import *

In [3]:
#
# 1. Definir datos
#
a, b = 0, 1 # Intervalo espacial de definición
nx = 40 # Número de subintervalos en espacio
x_i = np.linspace(a, b, nx) # Vector de puntos espaciales
dx = x_i[1]-x_i[0] # Distancia entre los puntos

def q0(x): # Población en el instante inicial
    return np.exp(-50*(x-0.3)**2)

nt = 10 # Número de iteraciones de tiempo
dt = 0.02 # Paso en tiempo

velocidad=1

#
# 2. Ejecutar el test y, eventualmente, comprobar los resultados
#
U = advection1D_Upwind_FVM (a, b, 
                            beta=velocidad, 
                            u0=q0(x_i), 
                            dx=dx, dt=dt, nt=nt,
                            verbosity=False
                           )

comprobar_resultado = False
if comprobar_resultado:
    for n in range(0,nt,nt//10):
        print("n =", n)
        mpl.plot(U[n], label=f"$u_{{{n}}}$")
        mpl.legend()
        mpl.show()

In [54]:
# Import widgets from ipywidgets
from ipywidgets import interact_manual, interact
import ipywidgets as widgets

# Matplotlib includes a function "animation"
from matplotlib import animation, rc

# And in Jupyter we can draw HTML
from IPython.display import HTML

# Definition of the animation
a, b = 0, 1  # Space interval: [a,b]
T = 1        # Time interval: [0,T]


variacion_total = 0
def animacion(beta, nx, nt):
    dt = T/nt
    C1=1; C2=50; C3=0
    R=1; G=0.48; B=0;
    width=2
    def q0(x): return (C1 * np.exp(-C2*(x-C3)**2))
    
    x_i = np.linspace(a,b,nx)
    dx=x_i[1]-x_i[0]
    U = advection1D_Upwind_FVM (a,b, beta, 
                            u0=q0(x_i), 
                            dx=dx, dt = dt, nt=nt,
                            verbosity=False)
    

    # The basis:


    # First set up the figure, the axis, and the plot element we want to animate
    fig, ax = mpl.subplots(figsize=(12,6))
    ax.set_xlim( (a, b) )
    #   ...we use the initial condition to prepare y-limits
    y_m = 1.1*min(U[0])
    y_M = 1.1*max(U[0])
    ax.set_ylim( (y_m, y_M) )
    ax.grid()
    x_text, y_text = 0*9*x_i[0]+0.1*x_i[-1], 0.5*(y_m+y_M)
    
    line, = ax.plot([], [], 'o-', lw=width, color=(R, G, B), markersize=3*width)
    #fig.set_size_inches(17, 9, forward=True)
    mpl.close(fig)

    # Initialization for each frame (it plots the background):
    def init():
        line.set_data([], [])
        return (line,)

    # Animation function, which is called for each new frame:
    def animate(i):
        y_i = U[i]
        line.set_data(x_i, y_i)
        ax.set_title(f"Etapa de tiempo {i+1} \n (tiempo {(i+1)*dt:.2f}, dt={dt:.2f})")
        if(i==nt-1):
            var_total = np.var(np.diff(U))/dx
            valor_inicial, valor_final = U[0][0], U[-1][-1]
            texto = f"Variación total: {var_total:.2f}"
            texto += f"\n\nValor final: {valor_final:.2f} (difusión: {valor_inicial-valor_final:.2f})"
            color="black"
            if var_total < 1:
                if valor_final > valor_inicial/2: color="green" 
            else:
                color="red"
                texto += "\n\n¡SISTEMA INESTABLE!"
            ax.text(x_text, y_text, texto, size="x-large", color=color)
        return (line,)

    # Compile the animation. Setting blit=True will only re-draw
    # the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=nt, interval=100, 
                                   blit=True, repeat=False)
    
    # And use JavaScript in HTML to show results
    return HTML(anim.to_jshtml())

nt_widget = widgets.BoundedIntText(value=60, min=1.0, max=200.0, step=1, description = "Iteraciones (tiempo)")
nx_widget = widgets.BoundedIntText(value=30, min=0.0, max=100.0, step=1, description = "Muestras en espacio")
dt_widget = widgets.FloatSlider(value=0.02, min=0.0, max=1.0, step=0.001, description = "Paso de t")
beta_widget = widgets.FloatSlider(value=1, min=0.0, max=5.0, step=0.01, description = r"$\beta$ (velocidad)")
widgets.interact_manual.opts['manual_name'] = 'Ejecutar simulación...'

interact_manual(animacion, nx = nx_widget, dt = dt_widget, nt = nt_widget, beta = beta_widget)

interactive(children=(FloatSlider(value=1.0, description='$\\beta$ (velocidad)', max=5.0, step=0.01), BoundedI…

<function __main__.animacion(beta, nx, nt)>

In [13]:
np.var(U)

0.10857848718371677